| 參數                                       | 說明                               | 是否推薦調整     | 影響層面            |
| ---------------------------------------- | -------------------------------- | ---------- | --------------- |
| `model`                                  | 使用的模型名稱（如 `"llama3:8b"`）         | 必須設      | 根本模型選擇          |
| `temperature`                            | 溫度，控制隨機性，0 = 最穩定，1 = 最具創意        | 可調      | 內容創意 vs 一致性     |
| `top_p`                                  | nucleus sampling，控制高機率詞的範圍（0\~1） | 可調      | 生成多樣性           |
| `top_k`                                  | 前 K 大機率詞彙中選擇（整數）                 | 可調         | 限制範圍選字          |
| `repeat_penalty`                         | 重複懲罰，防止重複詞句（建議 1.1\~1.5）         | 可調      | 內容品質、避免重複       |
| `num_predict`                            | 預測的 token 數量上限                   | 可調         | 回應長度上限          |
| `num_ctx`                                | context 長度（上下文 token 數），依模型決定最大值 | 若模型支援可調 | 記憶上下文能力         |
| `mirostat`、`mirostat_eta`、`mirostat_tau` | 動態調整溫度的算法參數                      | 高階用        | 保持語言平衡、控制熵值     |
| `stop`                                   | 停止生成的關鍵詞（如 `["</s>"]`）           | 可調         | 控制輸出結束時機        |
| `system`                                 | system prompt，設定 AI 的行為角色        | 非常推薦調整   | 明確任務導向（如：你是評論員） |
| `template`                               | 使用特定輸出模板（視模型支援）                  | 可調         | 定制格式            |
| `format`                                 | 設定回傳格式，如 `json`, `text`, `full`  | 若你要解析    | 結構化結果           |

| 參數                | 說明                                            |
| ----------------- | --------------------------------------------- |
| `base_url`        | 連線的 Ollama host（通常是 `http://localhost:11434`） |
| `num_thread`      | 使用的 CPU 執行緒數量，預設為 1，調大可加速                     |
| `num_gpu`         | 使用 GPU 數量，視本地支援                               |
| `timeout`         | 等待回應的秒數                                       |
| `keep_alive`      | 保持連線時間                                        |
| `raw`             | 若為 True，保留原始回傳格式（你要自己處理）                      |
| `headers`, `auth` | 若接 API，可設自定 headers 或認證                       |
| `verbose`         | 是否顯示詳細調用資訊                                    |


In [1]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.chat_models import ChatOllama  # 使用 Ollama 封裝的 LLaMA 模型
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
import torch


# 使用本地 LLaMA 模型
llm_llama3_8B = ChatOllama(model="llama3:8B")
llm_phi3_3dot8B = ChatOllama(model="phi3:3.8B")
llm_mistral_7B = ChatOllama(model="mistral:7B")

# GPU 加速
print("=== PyTorch GPU 加速環境檢查 ===")
print(f"PyTorch 版本: {torch.__version__}")
print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
print(f"是否支援 CUDA: {torch.cuda.is_available()}")

try:
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        current_gpu = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_gpu)

        print(f"偵測到 {gpu_count} 個 GPU")
        print(f"當前使用的 GPU：{device_name}")
        device = torch.device("cuda")
    else:
        raise RuntimeError("CUDA 不可用，將使用 CPU")
except Exception as e:
    print(f"無法使用 GPU：{e}")
    device = torch.device("cpu")

print(f"pytorch可用的GPU為：{device}")


c:\Users\USER\Documents\SideProj\base-agent\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


=== PyTorch GPU 加速環境檢查 ===
PyTorch 版本: 2.5.1+cu121
PyTorch 編譯的 CUDA 版本: 12.1
是否支援 CUDA: True
偵測到 1 個 GPU
當前使用的 GPU：NVIDIA GeForce RTX 3060 Ti
pytorch可用的GPU為：cuda


C:\Users\USER\AppData\Local\Temp\ipykernel_53512\2325541876.py:12: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm_llama3_8B = ChatOllama(model="llama3:8B")


### 建立工具 ###

In [ ]:
def say_hello(name: str) -> str:
    return f"哈囉，{name}，我是來自 LLaMA 的代理人！"

tools = [
    Tool(
        name="GreetTool",
        func=say_hello,
        description="用來打招呼的工具"
    )
]

### 建立 Agent ###

In [ ]:

agent_llama3_8B = initialize_agent(
    tools=tools,
    llm=llm_llama3_8B,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent_mistral_7B = initialize_agent(
    tools=tools,
    llm=llm_mistral_7B,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent_mistral_7B = initialize_agent(
    tools=tools,
    llm=llm_mistral_7B,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 測試 prompt
# response = agent_llama3_8B.run("請簡單介紹你自己是什麼樣的 Agent")
# print(response)

### 記憶 ###

In [ ]:
# IPython（Jupyter Notebook 的內核）提供的魔法指令（magic commands），用來自動重新載入外部 Python 模組
# 不用每次修改完外部 .py 檔案後都手動重新啟動 kernel 或重新 import
%load_ext autoreload
%autoreload 2

from memory import Memory, MemoryStream
from reflection import ReflectionEngine

memory_stream = MemoryStream()

# 新增一些記憶（模擬）
memory_stream.add_memory("今天與村民 Alice 討論了氣候變遷，發現她非常關心環境議題", importance=6)
memory_stream.add_memory("Alice 常常在社區清理垃圾", importance=7)
memory_stream.add_memory("Alice 主動發起社區紙類回收活動", importance=9)

# 啟動反思模組
reflector = ReflectionEngine(llm=llm_llama3_8B, memory_stream=memory_stream, reflection_threshold=20)
reflector.reflect()